### En un segundo scrapeo ya teniendo las URLs de cada uno de los festivales. Accederé a cada URL y me traeré los datos completos de cada festival de música (nombre, lugar, latitud, longitud, estilos de música, artistas, etc...)

## Importo la librerías necesarias para el Scrapping y almacenamiento/ tratamiento de los datos

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import json
import numpy as np
from datetime import datetime

## Inicializo el diccionario donde iré almacenando los datos para después crear el DataFrame

In [27]:
# Inicializo el diccionario

dicc_festivales = {
                    "festival": [],
                    "lugar": [],
                    "fecha_inicio": [],
                    "fecha_fin": [],
                    "duracion": [],
                    "cod_postal": [],
                    "latitud": [],
                    "longitud": [],
                    "provincia": [],
                    "localidad": [],
                    "gratuito": [],
                    "artistas": [],
                    "estilos": [],
                   }

Importo el archivo csv con las URL

In [4]:
lista_url = pd.read_csv(r"..\data\datos_scrapping\lista_url.csv", index_col="Unnamed: 0")
lista_url

,url
0,https://fanmusicfest.com//content/lovin-ibiza-...
1,https://fanmusicfest.com//content/ims-dalt-vil...
2,https://fanmusicfest.com//content/darkmad-2023
3,https://fanmusicfest.com//content/tv-hardcore-...
4,https://fanmusicfest.com//content/valencia-flo...
...,...
139,https://fanmusicfest.com//content/valladolid-b...
140,https://fanmusicfest.com//content/carrion-rock...
141,https://fanmusicfest.com//content/wachina-wach...
142,https://fanmusicfest.com//content/black-sound-...


In [6]:
url_list = list(lista_url["url"])
len(url_list)

545

In [24]:
def get_data(lista_URL):

    ''' 
    Función que conecta con la página web para extraer los datos de cada uno de los festivales y la almacena en un diccionario

    input: Recibe una lista (lst) de las URLs para cada estilo de música
    output: Retorna una diccionario (dict) con la información recogida de cada uno de los festivales
    
    '''
    for url in lista_URL:

        try:
            response = requests.get(url)
            html = response.content
            soup = bs(html, "lxml")

            geoJson = soup.find_all("script", {"type": "application/ld+json"})
            nombre = soup.find_all("h1", {"class": "page-title mb-0 mr-xl-3 text-center text-xl-left"})[0].get_text()

            dicc_festivales["festival"].append(nombre)
            dicc_festivales["gratuito"].append(json.loads(geoJson[0].getText())["@graph"][0]["isAccessibleForFree"])
            dicc_festivales["lugar"].append(json.loads(geoJson[0].getText())["@graph"][0]["location"].get("name","sin nombre"))

            dicc_festivales["fecha_inicio"].append(json.loads(geoJson[0].getText())["@graph"][0]["startDate"])
            dia_inicio = pd.to_datetime(json.loads(geoJson[0].getText())["@graph"][0]["startDate"])

            dicc_festivales["fecha_fin"].append(json.loads(geoJson[0].getText())["@graph"][0]["endDate"])
            dia_fin = pd.to_datetime(json.loads(geoJson[0].getText())["@graph"][0]["endDate"])

            dicc_festivales["duracion"].append(int(str((dia_fin - dia_inicio)).split(" ")[0])+1)

            dicc_festivales["cod_postal"].append(json.loads(geoJson[0].getText())["@graph"][0]["location"]["address"].get("postalCode", "sin cod_postal"))
            dicc_festivales["latitud"].append(json.loads(geoJson[0].getText())["@graph"][0]["location"]["geo"].get("latitude","sin latitud"))
            dicc_festivales["longitud"].append(json.loads(geoJson[0].getText())["@graph"][0]["location"]["geo"].get("longitude","sin longitud"))
            dicc_festivales["provincia"].append(json.loads(geoJson[0].getText())["@graph"][0]["location"]["address"].get("addressRegion","sin provincia"))
            dicc_festivales["localidad"].append(json.loads(geoJson[0].getText())["@graph"][0]["location"]["address"].get("addressLocality", "sin localidad"))
                   
            lista_estilos = []
            estilos = soup.find_all("a", {"class": "text-lowercase"})
            for estilo in estilos:
                lista_estilos.append(estilo.get_text())
            else:
                lista_estilos.append("sin estilo")
            dicc_festivales["estilos"].append(lista_estilos)

            artistas = []
            campos_artistas = []
            for data in json.loads(geoJson[0].getText())["@graph"][0]:
                campos_artistas.append(data)                
            if "performer" in campos_artistas:
                for artista in json.loads(geoJson[0].getText())["@graph"][0]["performer"]:            
                    artistas.append(artista["name"])
                dicc_festivales["artistas"].append(artistas)
            else:
                dicc_festivales["artistas"].append("sin artistas")
                

        except:
            pass
 
    return dicc_festivales

In [28]:
get_data(url_list)

{'festival': ['Lovin Ibiza Festival 2023',
  'IMS Dalt Vila 2023',
  'DarkMAD 2023',
  'Tv Hardcore Festival 2023',
  'Valencia Flow Fest 2023',
  'Ritmo Festival 2023',
  'Festival Días de Campo 2023',
  'Molan los 90 Zaragoza 2023',
  'Love the 90’s Valencia 2023',
  "Homenaje a la Ruta 90's & 2MIL Benidorm 2023",
  'Sónar 2023',
  'OFFSónar 2023',
  'O Son do Camiño 2023',
  'OffWeek Festival 2023',
  'Animal Sound Festival 2023',
  'Love the 90’s Madrid 2023',
  'A Summer Story 2023',
  "Love the Tuenti's 2023",
  'Ultra Beach Costa del Sol 2023',
  'Holika Festival 2023',
  'Maudes Festival 2023',
  'Weekend Beach Festival 2023',
  "Homenaje a la Ruta 90's & 2MIL Valencia 2023",
  'FIB 2023',
  'Fan Futura Fest 2023',
  'Monegros Desert Festival 2023',
  'Loona Summer Festival 2023',
  'Arenal Sound 2023',
  'Dreambeach Festival 2023',
  'Medusa Sun Beach Festival 2023',
  'Aquasella 2023',
  'The End of the World Festival 2023',
  'Love the 90’s Alicante 2023',
  'Parallel Festiv

Compruebo que todos los campos tienen la misma longitud para poder crear el dataframe

In [29]:
for key, value in dicc_festivales.items():
    print(key, len(value))

festival 545
lugar 545
fecha_inicio 545
fecha_fin 545
duracion 545
cod_postal 545
latitud 545
longitud 545
provincia 545
localidad 545
gratuito 545
artistas 545
estilos 545


Creo el dataframe

In [30]:
df_festivales = pd.DataFrame(dicc_festivales)
df_festivales

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,gratuito,artistas,estilos
0,Lovin Ibiza Festival 2023,Lío Ibiza,2023-04-28,2023-04-30,3,07800,38.913852,1.443275,Islas Baleares,Eivissa,False,sin artistas,"[dj, techno, house, sin estilo]"
1,IMS Dalt Vila 2023,"[Baluarte de Santa Lucía, Dalt Vila]",2023-04-28,2023-04-28,1,07800,38.908434,1.438789,Islas Baleares,Ibiza,False,"[CamelPhat, Ida Engberg, Kölsch, Pete Tong, AN...","[electronic, techno, house, dance, sin estilo]"
2,DarkMAD 2023,Autocine,2023-04-29,2023-04-30,2,28034,40.486224,-3.677802,Madrid,Madrid,False,"[The Human League, Ana Curra, Blac Kolor, Pink...","[electro, post-punk, industrial, gothic metal,..."
3,Tv Hardcore Festival 2023,Family Club,2023-04-29,2023-04-29,1,45100,0.000000,0.000000,Toledo,Sonseca,False,"[Hardsuel, Pawlowski, DXPE, Dvrk Oktopus, Emei...","[techno, hardcore, hardstyle, sin estilo]"
4,Valencia Flow Fest 2023,La 3 park,2023-05-06,2023-05-07,2,46012,39.410957,-0.334871,Valencia,Pinedo,False,"[Ojete Calor, Mueveloreina, Rocío Sáiz, Kika L...","[pop, electronic, techno, house, sin estilo]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Valladolid Blues 2023,Teatro Carrión,2023-02-10,2023-03-10,29,47001,41.649996,-4.728177,Valladolid,Valladolid,False,"[Martín Burguez Trío, Mingo Balaguer, Travelli...","[blues, sin estilo]"
541,Carrion Rock Festival 2023,Sala Infinity,2023-02-11,2023-02-11,1,34004,42.006608,-4.519082,Palencia,Palencia,False,"[Guadaña, Ciclón, Invadeath]","[rock, metal, heavy metal, sin estilo]"
542,Wáchina Wáchina Fest 2023,16 Toneladas,2023-02-18,2023-02-18,1,46009,39.481255,-0.387633,Valencia,Valencia,False,"[The Mourning After, The Mocks, MOOON, The Cou...","[punk rock, rock n roll, garage rock, sin estilo]"
543,Black Sound Fest 2023,Teatro Imperial,2023-02-04,2023-02-25,22,06400,38.953452,-5.860906,Badajoz,Don Benito,False,"[Vargas Blues Band, Gonzalo Portugal, The Flam...","[rock, blues, sin estilo]"


Exporto a csv para realizar un posterior análisis

In [32]:
df_festivales.to_csv(f"..\data\datos_demograficos\df_festivales_{len(df_festivales)}.csv", encoding= "utf16")